In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy import stats
import itertools
import re
import matplotlib.pyplot as plt
import seaborn as sns
import helper

In [31]:
import requests
from bs4 import BeautifulSoup as soup

In [3]:
hous_df = pd.read_csv('Ames_Housing_Price_Data.csv', 
                      index_col='PID', low_memory=False)
hous_df = hous_df.drop('Unnamed: 0', axis=1)
hous_df = hous_df.drop_duplicates()

In [4]:
mark_df = pd.read_csv('Ames_Real_Estate_Data.csv', 
                      index_col='MapRefNo', low_memory=False)
mark_df = mark_df.drop_duplicates()

In [13]:
hm_df = hous_df.merge(mark_df, how='inner', left_index=True, right_index=True, indicator=True)

In [16]:
hm_df.shape

(2602, 171)

In [70]:
add = [add + ', AMES, IA' for add in hm_df['Prop_Addr']]
add = ['https://www.zillow.com/homes/' + s.replace(' ', '-') + '_rb/' for s in add]
adds = pd.DataFrame(index=hm_df.index, data={'www':add}, dtype='string')

In [72]:
adds['www']

526301100     https://www.zillow.com/homes/3126-NORTHWOOD-DR...
526302030     https://www.zillow.com/homes/3115-NORTHWOOD-DR...
526302040     https://www.zillow.com/homes/3113-NORTHWOOD-DR...
526302110     https://www.zillow.com/homes/623-RIVER-OAK-DR,...
526302120     https://www.zillow.com/homes/627-RIVER-OAK-DR,...
                                    ...                        
924100070     https://www.zillow.com/homes/3508-JEWEL-DR,-AM...
924151040     https://www.zillow.com/homes/3616-JEWEL-DR,-AM...
924151050     https://www.zillow.com/homes/3702-JEWEL-DR,-AM...
924152030     https://www.zillow.com/homes/3715-JEWEL-DR,-AM...
1007100110    https://www.zillow.com/homes/2520-E-LINCOLN-WA...
Name: www, Length: 2602, dtype: string

In [74]:
adds.iloc[1,0]

'https://www.zillow.com/homes/3115-NORTHWOOD-DR,-AMES,-IA_rb/'

In [32]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [76]:
p_ = 'https://www.zillow.com/homes/2322-BUCHANAN-DR,-AMES,-IA_rb/'
r_ = requests.get(p_, headers = headers)
s_ = soup(r_.text, 'html.parser')
d_ = s_.find_all('span', attrs = {'class': 'Text-c11n-8-38-0__aiai24-0 jtMauM'})
# h_ = [tag.find('a').get('title') for tag in d_]
d_

[]

In [46]:
zest_info = []
for link in adds['www']:
    r_ = requests.get(link, headers = headers)
    s_ = soup(r_.text, 'html.parser')
    d_ = s_.find_all('span', attrs = {'class': 'Text-c11n-8-38-0__aiai24-0 jtMauM'})
    zest_info.append(d_)

In [75]:
link = 'https://www.zillow.com/homes/3115-NORTHWOOD-DR,-AMES,-IA_rb/'
r_ = requests.get(link, headers = headers)
s_ = soup(r_.text, 'html.parser')
d_ = s_.find_all('span', attrs = {'class': 'Text-c11n-8-38-0__aiai24-0 jtMauM'})
d_

[]

In [47]:
adds['zill'] = zest_info

In [49]:
zest_info

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [<span class="Text-c11n-8-38-0__aiai24-0 jtMauM">$265,900</span>,
  <span class="Text-c11n-8-38-0__aiai24-0 jtMauM">$1,300/mo</span>,
  <span class="Text-c11n-8-38-0__aiai24-0 jtMauM">Est. total selling costs (<!-- -->10<!-- -->%)</span>,
  <span class="Text-c11n-8-38-0__aiai24-0 jtMauM">$26,751</span>,
  <span class="Text-c11n-8-38-0__aiai24-0 jtMauM">$1,300</span>,
  <span class="Text-c11n-8-38-0__aiai24-0 jtMauM">$1,300</span>,
  <span class="Text-c11n-8-38-0__aiai24-0 jtMauM">$1,282</span>,
  <span class="Text-c11n-8-38-0__aiai24-0 jtMauM">$1,282</span>,
  <span class="Text-c11n-8-38-0__aiai24-0 jtMauM">$273,700</span>,
  <span class="Text-c11n-8-38-0_

In [48]:
adds

,www,zill
526301100,https://www.zillow.com/homes/3126-NORTHWOOD-DR...,[]
526302030,https://www.zillow.com/homes/3115-NORTHWOOD-DR...,[]
526302040,https://www.zillow.com/homes/3113-NORTHWOOD-DR...,[]
526302110,"https://www.zillow.com/homes/623-RIVER-OAK-DR,...",[]
526302120,"https://www.zillow.com/homes/627-RIVER-OAK-DR,...",[]
...,...,...
924100070,"https://www.zillow.com/homes/3508-JEWEL-DR,-AM...",[]
924151040,"https://www.zillow.com/homes/3616-JEWEL-DR,-AM...",[]
924151050,"https://www.zillow.com/homes/3702-JEWEL-DR,-AM...",[]
924152030,"https://www.zillow.com/homes/3715-JEWEL-DR,-AM...",[]
